# Build Vector Database from PDF
This notebook processes PDF files and creates a FAISS vector database for RAG applications.

In [1]:
# Install required packages
!pip install langchain pypdf2 faiss-cpu sentence-transformers

In [2]:
import os
from typing import List
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [9]:
from PyPDF2 import PdfReader

def extract_toc(pdf_path):
    reader = PdfReader(pdf_path)
    if hasattr(reader, "outline"):
        toc = reader.outline
        return toc
    return None

pdf_path = "nlp_book.pdf"
toc = extract_toc(pdf_path)
for i in toc:
    print(i)


{'/Title': 'I Fundamental Algorithms for NLP', '/Page': IndirectObject(2156, 0, 124214675915072), '/Type': '/XYZ', '/Left': 160.2, '/Top': 665.681, '/Zoom': NullObject, '/Count': -12}
[{'/Title': 'Introduction', '/Page': IndirectObject(2162, 0, 124214675915072), '/Type': '/XYZ', '/Left': 160.2, '/Top': 682.844, '/Zoom': NullObject}, {'/Title': 'Regular Expressions, Tokenization, Edit Distance', '/Page': IndirectObject(2171, 0, 124214675915072), '/Type': '/XYZ', '/Left': 142.2, '/Top': 682.844, '/Zoom': NullObject, '/Count': -11}, [{'/Title': 'Regular Expressions', '/Page': IndirectObject(2179, 0, 124214675915072), '/Type': '/XYZ', '/Left': 160.2, '/Top': 474.815, '/Zoom': NullObject, '/Count': -7}, [{'/Title': 'Basic Regular Expression Patterns', '/Page': IndirectObject(2179, 0, 124214675915072), '/Type': '/XYZ', '/Left': 160.2, '/Top': 177.454, '/Zoom': NullObject}, {'/Title': 'Disjunction, Grouping, and Precedence', '/Page': IndirectObject(2200, 0, 124214675915072), '/Type': '/XYZ', 

In [11]:
from PyPDF2 import PdfReader

def parse_toc(toc, results=None, level=0):
    """
    Đệ quy duyệt qua mục lục PDF và lưu các tiêu đề và số trang vào danh sách kết quả.

    Args:
        toc: Mục lục PDF, thường là một danh sách các dictionary hoặc danh sách lồng nhau.
        results: Danh sách chứa kết quả (mặc định là None, sẽ khởi tạo một danh sách mới).
        level: Mức độ lồng nhau (cấp độ của mục lục).

    Returns:
        results: Danh sách các mục lục với tiêu đề và số trang.
    """
    if results is None:
        results = []

    for item in toc:
        if isinstance(item, dict):  # Mục đơn
            title = item.get('/Title', 'Unknown Title')
            page = item.get('/Page', None)
            if page:  # Nếu có trang, giải mã số trang
                page_number = page.idnum - 1  # Chuyển đổi IndirectObject sang số trang
            else:
                page_number = None
            results.append({'title': title, 'page': page_number, 'level': level})
        elif isinstance(item, list):  # Mục lồng nhau
            parse_toc(item, results, level + 1)

    return results


def extract_toc_with_pages(pdf_path):
    """
    Trích xuất mục lục từ file PDF và giải mã tiêu đề, số trang.

    Args:
        pdf_path: Đường dẫn tới file PDF.

    Returns:
        List: Danh sách các mục lục với tiêu đề và số trang.
    """
    reader = PdfReader(pdf_path)

    if hasattr(reader, "outline"):
        toc = reader.outline
        results = parse_toc(toc)
        return results
    else:
        print("Không tìm thấy mục lục trong file PDF.")
        return None


# Đường dẫn tới file PDF
pdf_path = "nlp_book.pdf"

# Gọi hàm để trích xuất mục lục
toc_results = extract_toc_with_pages(pdf_path)

# Hiển thị kết quả
if toc_results:
    for entry in toc_results:
        indent = "  " * entry['level']  # Tạo thụt lề theo cấp độ
        print(f"{indent}Title: {entry['title']}, Page: {entry['page']}")


Title: I Fundamental Algorithms for NLP, Page: 2155
  Title: Introduction, Page: 2161
  Title: Regular Expressions, Tokenization, Edit Distance, Page: 2170
    Title: Regular Expressions, Page: 2178
      Title: Basic Regular Expression Patterns, Page: 2178
      Title: Disjunction, Grouping, and Precedence, Page: 2199
      Title: A Simple Example, Page: 2203
      Title: More Operators, Page: 2214
      Title: A More Complex Example, Page: 2222
      Title: Substitution, Capture Groups, and ELIZA, Page: 2222
      Title: Lookahead Assertions, Page: 2250
    Title: Words, Page: 2250
    Title: Corpora, Page: 2290
    Title: Simple Unix Tools for Word Tokenization, Page: 2325
    Title: Word and Subword Tokenization, Page: 2334
      Title: Top-down (rule-based) tokenization, Page: 2342
      Title: Byte-Pair Encoding: A Bottom-up Tokenization Algorithm, Page: 2390
    Title: Word Normalization, Lemmatization and Stemming, Page: 2407
      Title: Lemmatization, Page: 2407
    Title: Se

In [15]:
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

def extract_toc_and_text(pdf_path):
    """
    Trích xuất mục lục và nội dung từ file PDF.

    Args:
        pdf_path: Đường dẫn tới file PDF.

    Returns:
        List: Danh sách các mục lục với tiêu đề, số trang bắt đầu và nội dung.
    """
    reader = PdfReader(pdf_path)
    toc_data = []

    # Trích xuất mục lục
    if hasattr(reader, "outline"):
        outline = reader.outline

        for item in outline:
            if isinstance(item, dict):
                title = item.get('/Title', 'Unknown Title')
                page_ref = item.get('/Page')
                if page_ref:
                    # Lấy số trang thực tế từ đối tượng IndirectObject
                    start_page = reader.get_page_number(page_ref) + 1  # Số trang bắt đầu từ 1
                    toc_data.append({"title": title, "start_page": start_page})
    else:
        print("Không tìm thấy mục lục trong file PDF.")
        return None

    # Trích xuất nội dung cho từng mục lục
    for i, toc_item in enumerate(toc_data):
        start_page = toc_item["start_page"] - 1  # Chuyển về chỉ số bắt đầu từ 0
        end_page = toc_data[i + 1]["start_page"] - 1 if i + 1 < len(toc_data) else len(reader.pages)

        # Ghép nội dung từ các trang thuộc mục lục này
        text = ""
        for page_num in range(start_page, end_page):
            text += reader.pages[page_num].extract_text()
        toc_item["text"] = text

    return toc_data

def build_combined_index(pdf_path: str, index_path: str):
    """
    Xây dựng một vector store duy nhất cho toàn bộ nội dung PDF, được chia nhỏ theo mục lục lớn và nhỏ.

    Args:
        pdf_path: Đường dẫn tới file PDF.
        index_path: Đường dẫn lưu chỉ mục.
    """
    # Initialize embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    # Extract TOC and associated text
    toc_data = extract_toc_and_text(pdf_path)
    if not toc_data:
        print("Không có mục lục để xử lý.")
        return

    # Initialize vector store
    combined_texts = []

    for toc_item in toc_data:
        title = toc_item["title"]
        text = toc_item["text"]

        # Split text into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        chunks = text_splitter.split_text(text)

        # Append title to each chunk for context
        titled_chunks = [f"{title}: {chunk}" for chunk in chunks]
        combined_texts.extend(titled_chunks)

    # Create and save vector store
    vector_store = FAISS.from_texts(combined_texts, embeddings)
    vector_store.save_local(index_path)
    print(f"Combined index saved to {index_path}")

# Example usage
pdf_path = "nlp_book.pdf"
index_path = "faiss_combined_index"
build_combined_index(pdf_path, index_path)


Combined index saved to faiss_combined_index
